# Example using sklearn pipelines with ta library

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler

import sys
sys.path.append("..") # Adds higher directory to python modules path.
from ta import *

In [2]:
# SETTINGS

LABELING = 'binary' # 'regression' or 'binary'
target = 'target' # name target column

if LABELING == 'regression':
    score = "neg_mean_absolute_error"
    model = ExtraTreesRegressor()
elif LABELING == 'binary':
    score = "accuracy"
    model = ExtraTreesClassifier()

In [3]:
# Load data
df = pd.read_csv('../data/datas.csv', sep=',')

# Nans values
df = utils.dropna(df)

In [4]:
# Labeling
if LABELING == "regression":
    df[target] = (df['Close'] / df['Close'].shift(1)) - 1

elif LABELING == "binary":
    df[target] = 0 # 'KEEP & DOWN'
    df.loc[df['Close'] < df['Close'].shift(-1), target] = 1 # 'UP'

In [5]:
df = df.dropna()

In [6]:
cols = [i for i in df.columns if i != target]
X = df[cols]
y = df[target]

N = 500

X_train = df[cols][:-N]
y_train = df.target[:-N]
X_test = df[cols][-N:]
y_test = df.target[-N:]

In [7]:
pipe = make_pipeline(
    TAFeaturesTransform("Open", "High", "Low", "Close", "Volume_BTC", fillna=True),
    StandardScaler(),
    model
)

In [8]:
y_pred = pipe.fit(X_train, y_train).predict(X_test)

if LABELING == "regression":
    print(mean_absolute_error(y_test, y_pred))
elif LABELING == 'binary':
    print(accuracy_score(y_test, y_pred))

../ta/trend.py:205: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (dip_mio[i]/trs[i])
../ta/trend.py:209: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (din_mio[i]/trs[i])
../ta/trend.py:670: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  aroon_up = close.rolling(n).apply(lambda x: float(np.argmax(x) + 1) / n * 100)
../ta/trend.py:692: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  aroon_down = close.rolling(n).apply(lambda x: float(np.argmin(x) + 1) / n * 100)
/Users/lecrintech/projec

0.478


../ta/trend.py:670: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  aroon_up = close.rolling(n).apply(lambda x: float(np.argmax(x) + 1) / n * 100)
../ta/trend.py:692: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  aroon_down = close.rolling(n).apply(lambda x: float(np.argmin(x) + 1) / n * 100)


In [9]:
resultados = cross_val_score(pipe,
                            X = df[cols],
                            y = df[target],
                            scoring=score)
print(resultados)

/Users/lecrintech/projects/python/ta/env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
../ta/wrapper.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[volume], fillna=fillna)
../ta/wrapper.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['volume_obv'] = on_balance_volume(df[close], df[volume], fillna=fillna)
../ta/w

[0.51684374 0.4781989  0.51247166]


../ta/wrapper.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['others_dlr'] = daily_log_return(df[close], fillna=fillna)
../ta/wrapper.py:170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['others_cr'] = cumulative_return(df[close], fillna=fillna)
